# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 1 2022 3:25PM,249823,10,0086134506,ISDIN Corporation,Released
1,Nov 1 2022 3:25PM,249823,10,0086134511,ISDIN Corporation,Released
2,Nov 1 2022 3:25PM,249823,10,0086134509,ISDIN Corporation,Released
3,Nov 1 2022 3:25PM,249823,10,0086134513,ISDIN Corporation,Released
4,Nov 1 2022 3:25PM,249823,10,0086134510,ISDIN Corporation,Released
5,Nov 1 2022 3:25PM,249823,10,0086134512,ISDIN Corporation,Released
6,Nov 1 2022 3:25PM,249823,10,0086134507,ISDIN Corporation,Released
7,Nov 1 2022 3:25PM,249823,10,0086134508,ISDIN Corporation,Released
8,Nov 1 2022 3:25PM,249823,10,0086134515,ISDIN Corporation,Released
9,Nov 1 2022 3:25PM,249823,10,0086134516,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,249822,Released,2
15,249823,Released,24
16,249824,Released,4
17,249825,Released,1
18,249826,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249822,NaN,NaN,2.0
249823,NaN,NaN,24.0
249824,NaN,NaN,4.0
249825,NaN,NaN,1.0
249826,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249750,57.0,1.0,1.0
249757,0.0,0.0,1.0
249793,0.0,0.0,1.0
249794,0.0,0.0,1.0
249795,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249750,57,1,1
249757,0,0,1
249793,0,0,1
249794,0,0,1
249795,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249750,57,1,1
1,249757,0,0,1
2,249793,0,0,1
3,249794,0,0,1
4,249795,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249750,57,1,1
1,249757,,,1
2,249793,,,1
3,249794,,,1
4,249795,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation
24,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc."
26,Nov 1 2022 3:20PM,249825,22,"NBTY Global, Inc."
27,Nov 1 2022 3:19PM,249824,10,Emerginnova
31,Nov 1 2022 3:15PM,249822,10,"Snap Medical Industries, LLC"
33,Nov 1 2022 3:05PM,249820,10,ISDIN Corporation
37,Nov 1 2022 3:04PM,249819,10,Yusen – 3D Matrix
38,Nov 1 2022 2:25PM,249816,19,ACG North America LLC
41,Nov 1 2022 1:39PM,249807,19,"AdvaGen Pharma, Ltd"
56,Nov 1 2022 12:34PM,249798,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation,,,24
1,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc.",,,2
2,Nov 1 2022 3:20PM,249825,22,"NBTY Global, Inc.",,,1
3,Nov 1 2022 3:19PM,249824,10,Emerginnova,,,4
4,Nov 1 2022 3:15PM,249822,10,"Snap Medical Industries, LLC",,,2
5,Nov 1 2022 3:05PM,249820,10,ISDIN Corporation,,,4
6,Nov 1 2022 3:04PM,249819,10,Yusen – 3D Matrix,,,1
7,Nov 1 2022 2:25PM,249816,19,ACG North America LLC,,,3
8,Nov 1 2022 1:39PM,249807,19,"AdvaGen Pharma, Ltd",,,15
9,Nov 1 2022 12:34PM,249798,12,Hush Hush,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation,24,,
1,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc.",2,,
2,Nov 1 2022 3:20PM,249825,22,"NBTY Global, Inc.",1,,
3,Nov 1 2022 3:19PM,249824,10,Emerginnova,4,,
4,Nov 1 2022 3:15PM,249822,10,"Snap Medical Industries, LLC",2,,
5,Nov 1 2022 3:05PM,249820,10,ISDIN Corporation,4,,
6,Nov 1 2022 3:04PM,249819,10,Yusen – 3D Matrix,1,,
7,Nov 1 2022 2:25PM,249816,19,ACG North America LLC,3,,
8,Nov 1 2022 1:39PM,249807,19,"AdvaGen Pharma, Ltd",15,,
9,Nov 1 2022 12:34PM,249798,12,Hush Hush,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation,24,,
1,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc.",2,,
2,Nov 1 2022 3:20PM,249825,22,"NBTY Global, Inc.",1,,
3,Nov 1 2022 3:19PM,249824,10,Emerginnova,4,,
4,Nov 1 2022 3:15PM,249822,10,"Snap Medical Industries, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation,24,NaN,NaN
1,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc.",2,NaN,NaN
2,Nov 1 2022 3:20PM,249825,22,"NBTY Global, Inc.",1,NaN,NaN
3,Nov 1 2022 3:19PM,249824,10,Emerginnova,4,NaN,NaN
4,Nov 1 2022 3:15PM,249822,10,"Snap Medical Industries, LLC",2,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation,24,0.0,0.0
1,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc.",2,0.0,0.0
2,Nov 1 2022 3:20PM,249825,22,"NBTY Global, Inc.",1,0.0,0.0
3,Nov 1 2022 3:19PM,249824,10,Emerginnova,4,0.0,0.0
4,Nov 1 2022 3:15PM,249822,10,"Snap Medical Industries, LLC",2,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1249108,35,0.0,0.0
12,249798,1,0.0,0.0
15,249750,1,1.0,57.0
16,249757,1,0.0,0.0
19,999246,21,10.0,0.0
21,749382,3,0.0,0.0
22,249825,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1249108,35,0.0,0.0
1,12,249798,1,0.0,0.0
2,15,249750,1,1.0,57.0
3,16,249757,1,0.0,0.0
4,19,999246,21,10.0,0.0
5,21,749382,3,0.0,0.0
6,22,249825,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,35,0.0,0.0
1,12,1,0.0,0.0
2,15,1,1.0,57.0
3,16,1,0.0,0.0
4,19,21,10.0,0.0
5,21,3,0.0,0.0
6,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,35.0
1,12,Released,1.0
2,15,Released,1.0
3,16,Released,1.0
4,19,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,0.0,0.0,57.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,1.0,0.0,10.0,0.0,0.0
Released,35.0,1.0,1.0,1.0,21.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,57.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,1.0,0.0,10.0,0.0,0.0
2,Released,35.0,1.0,1.0,1.0,21.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,57.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,1.0,0.0,10.0,0.0,0.0
2,Released,35.0,1.0,1.0,1.0,21.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()